## ETL Notebook for CA Biositing Project

This notebook provides a documented walkthrough of the ETL (Extract, Transform, Load) process for the CA Biositing project. It is designed for interactive development and exploration before migrating logic into the production pipeline.

It covers:

1.  **Setup**: Importing necessary libraries and establishing a connection to the database.
2.  **Extraction**: Pulling raw data from Google Sheets.
3.  **Cleaning**: Standardizing data types, handling missing values, and cleaning column names.
4.  **Normalization**: Replacing human-readable names (e.g., "Corn") with database foreign key IDs (e.g., `resource_id: 1`).
5.  **Utilities**: Common functions for data manipulation and analysis.
6.  **Deployment Plan**: A step-by-step guide for moving the code from this notebook into the production ETL modules.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import janitor as jn
import logging
from IPython.display import display
from sqlalchemy.orm import Session
from sqlalchemy import select

# --- Basic Logging Configuration for Notebook ---
# When running in a notebook, we use Python's standard logging.
# In the production pipeline, this will be replaced by Prefect's `get_run_logger()`
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# --- Import project modules ---
# All namespace packages are installed in the Pixi environment.
# Use the Pixi kernel in Jupyter to ensure correct imports.
try:
    from ca_biositing.pipeline.utils.engine import engine
    from ca_biositing.datamodels.schemas.generated.ca_biositing import *
    from ca_biositing.pipeline.utils.name_id_swap import replace_name_with_id_df
    from ca_biositing.pipeline.etl.extract import proximate, ultimate, cmpana, samplemetadata
    logger.info('Successfully imported all project modules.')
except ImportError as e:
    logger.error(f'Failed to import project modules: {e}', exc_info=True)

### TRANSFORMATION FUNCTIONS

### Data Cleaning Function

In [ ]:
# Use the refactored cleaning/coercion helpers from the new package
from ca_biositing.pipeline.utils.cleaning_functions import standard_clean, coerce_columns, coerce_columns_list

def clean_the_gsheets(df, lowercase=True, replace_empty=True):
    """Wrapper that applies the standardized cleaning pipeline implemented in `cleaning_functions`."""
    logger.info('Starting DataFrame cleaning via standard_clean.')
    if not isinstance(df, pd.DataFrame):
        logger.error('Input is not a pandas DataFrame.')
        return None
    try:
        # Run the composed standard clean (names, empty->NA, lowercase, convert_dtypes)
        df_cleaned = standard_clean(df, lowercase=lowercase, replace_empty=replace_empty)
        # Preserve behaviour: drop rows missing key columns if present
        subset = [c for c in ['resource', 'value'] if c in df_cleaned.columns]
        if subset:
            df_cleaned = df_cleaned.dropna(subset=subset)
        logger.info(f'Cleaning complete; rows remaining: {len(df_cleaned)}')
        return df_cleaned
    except Exception as e:
        logger.error(f'An error occurred during DataFrame cleaning: {e}', exc_info=True)
        return None


# --- Coercion Configuration Templates ---
# You can define column coercions in two ways: explicit keyword arguments or a dtype_map dictionary.
# For geometry: use geopandas to load shapefiles (geometry column is already properly typed).
# Only use geometry_cols if you have WKT strings to parse.

# APPROACH 1: Explicit keyword arguments (clear and direct)
COERCION_CONFIG_EXPLICIT = {
    'int_cols': ['repl_no', 'sample_no'],
    'float_cols': ['value', 'measurement'],
    'datetime_cols': ['created_at', 'updated_at'],
    'bool_cols': ['is_valid'],
    'category_cols': ['status'],
    'geometry_cols': []  # Use only if you have WKT strings; prefer geopandas for shapefiles
}

# APPROACH 2: dtype_map dictionary (compact, useful for dynamic configs)
COERCION_CONFIG_DTYPE_MAP = {
    'int': ['repl_no', 'sample_no'],
    'float': ['value', 'measurement'],
    'datetime': ['created_at', 'updated_at'],
    'bool': ['is_valid'],
    'category': ['status'],
    'geometry': []  # Use only if you have WKT strings; prefer geopandas for shapefiles
}

# APPROACH 3: GeoPandas GeoDataFrame (for shapefiles and spatial data)
# When loading shapefiles with geopandas, geometry is already a GeoSeries.
# Use geometry_format='geodataframe' to skip coercion:
GEOPANDAS_CONFIG = {
    'int_cols': ['id', 'repl_no'],
    'float_cols': ['area', 'value'],
    'geometry_cols': ['geometry'],
    'geometry_format': 'geodataframe'  # Don't convert; already properly typed
}

# Usage: coerce_the_gsheets(df, **COERCION_CONFIG_EXPLICIT)
#   or: coerce_the_gsheets(df, dtype_map=COERCION_CONFIG_DTYPE_MAP)
#   or: coerce_the_gsheets(gdf, **GEOPANDAS_CONFIG)  # for GeoDataFrames


def coerce_the_gsheets(df, dtype_map=None, int_cols=None, float_cols=None, datetime_cols=None, bool_cols=None, category_cols=None, geometry_cols=None, geometry_format='wkt'):
    """Coerce specified columns on a cleaned DataFrame using coercion helpers.
    `dtype_map` is an alternative mapping where keys are 'int','float','datetime','bool','category','geometry'.
    `geometry_format` controls geometry coercion: 'wkt' (parse WKT strings) or 'geodataframe' (skip, already typed)."""
    if not isinstance(df, pd.DataFrame):
        logger.error('coerce_the_gsheets: input is not a DataFrame')
        return df
    return coerce_columns(df, int_cols=int_cols, float_cols=float_cols, datetime_cols=datetime_cols, bool_cols=bool_cols, category_cols=category_cols, geometry_cols=geometry_cols, dtype_map=dtype_map, geometry_format=geometry_format)

### GeoPandas: Working with Shapefiles

For spatial data (shapefiles, GeoJSON, etc.), use **geopandas** instead of shapely string parsing. GeoPandas provides:
- Direct shapefile loading into GeoDataFrames
- Geometry column automatically typed as `geometry`
- Spatial operations (spatial joins, intersects, buffer, etc.)
- Better integration with tabular data

**Loading shapefiles:**
```python
import geopandas as gpd

# Load a shapefile directly (geometry column is already correct type)
gdf = gpd.read_file('path/to/shapefile.shp')

# Clean tabular columns (geometry is preserved)
gdf_clean = clean_the_gsheets(gdf)

# Coerce columns, skipping geometry since it's already a GeoSeries
gdf_coerced = coerce_the_gsheets(
    gdf_clean,
    int_cols=['id', 'repl_no'],
    float_cols=['area', 'value'],
    datetime_cols=['created_at'],
    geometry_cols=['geometry'],
    geometry_format='geodataframe'  # geometry is already properly typed
)
```


### Data Normalization Function

In [ ]:
def normalize_dataframes(dataframes, normalize_columns):
    """Normalizes a list of DataFrames by replacing name columns with foreign key IDs.

    This function iterates through a list of dataframes and, for each one, iterates
    through a dictionary of columns that need to be normalized. It uses the 
    `replace_name_with_id_df` utility to look up or create the corresponding ID
    in the database.

    Args:
        dataframes (list[pd.DataFrame]): A list of DataFrames to normalize.
        normalize_columns (dict): A dictionary mapping column names to SQLModel classes and attributes.

    Returns:
        list[pd.DataFrame]: The list of normalized DataFrames.
    """
    logger.info(f'Starting normalization process for {len(dataframes)} dataframes.')
    normalized_dfs = []
    try:
        with Session(engine) as db:
            for i, df in enumerate(dataframes):
                if not isinstance(df, pd.DataFrame):
                    logger.warning(f'Item {i+1} is not a DataFrame, skipping.')
                    continue
                
                logger.info(f'Processing DataFrame #{i+1} with {len(df)} rows.')
                df_normalized = df.copy()

                for df_col, (model, model_name_attr) in normalize_columns.items():
                    if df_col not in df_normalized.columns:
                        logger.warning(f"Column '{df_col}' not in DataFrame #{i+1}. Skipping normalization for this column.")
                        continue
                    
                    try:
                        # Skip normalization if the column is all NaN/None
                        if df_normalized[df_col].isnull().all():
                            logger.info(f"Skipping normalization for column '{df_col}' as it contains only null values.")
                            continue
                            
                        logger.info(f"Normalizing column '{df_col}' using model '{model.__name__}'.")
                        df_normalized, num_created = replace_name_with_id_df(
                            db=db,
                            df=df_normalized,
                            ref_model=model,
                            df_name_column=df_col,
                            model_name_attr=model_name_attr,
                            id_column_name='id',
                            final_column_name=f'{df_col}_id'
                        )
                        if num_created > 0:
                            logger.info(f"Created {num_created} new records in '{model.__name__}' table.")
                        new_col_name = f'{df_col}_id'
                        num_nulls = df_normalized[new_col_name].isnull().sum()
                        logger.info(f"Successfully normalized '{df_col}'. New column '{new_col_name}' contains {num_nulls} null values.")
                    except Exception as e:
                        logger.error(f"Error normalizing column '{df_col}' in DataFrame #{i+1}: {e}", exc_info=True)
                        continue # Continue to the next column
                
                normalized_dfs.append(df_normalized)
                logger.info(f'Finished processing DataFrame #{i+1}.')
            
            logger.info('Committing database session.')
            db.commit()
            logger.info('Database commit successful.')
    except Exception as e:
        logger.error(f'A critical error occurred during the database session: {e}', exc_info=True)
        db.rollback()
        logger.info('Database session rolled back.')
        
    return normalized_dfs


### ETL Execution Example

In [ ]:
# Geospatial cleaning example: standardize all lat/lon columns at once
from ca_biositing.pipeline.utils.cleaning_functions import (
    standardize_latlon,
    detect_latlon_columns,
)

# Example DataFrame with mixed lat/lon formats
sample_geo_df = pd.DataFrame({
    'site_id': [1, 2, 3, 4, 5],
    'sampling_lat': [40.7128, 34.0522, 37.7749, '41.8781', None],
    'sampling_lon': [-74.0060, -118.2437, -122.4194, '-87.6298', -120.0],
    'prod_location': ['40.5,-74.0', '34.2,-118.5', None, '', '37.5,-122.5'],
    'name': ['NYC', 'LA', 'SF', 'Chicago', 'Sacramento'],
})

print("Before standardization:")
df


# Auto-detect all lat/lon columns and standardize
sample_geo_standardized = standardize_latlon(
    df,
    auto_detect=True,
    output_lat='lat',
    output_lon='lon',
    coerce_to_float=True
)

print("\nAfter standardization:")
print(f"\nData types: {sample_geo_standardized[['lat', 'lon']].dtypes.to_dict()}")
sample_geo_standardized

In [ ]:
df = normalized_dataframes[0]

In [ ]:
df